In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

In [ ]:
dotlowell = pd.read_excel("/content/drive/MyDrive/CFJJ/Team 2/Lowell Data Files/part 1/Edited | DOTLowell_April2021.xlsx")
location = pd.read_excel("/content/drive/MyDrive/CFJJ/Team 2/Lowell Data Files/part 1/Edited | Location Analysis Report - April 2021.xlsx")

In [ ]:
#make the time count in 24h
for index in range(len(dotlowell)):
    if(dotlowell['AM-PM'][index]== "PM"):
        dotlowell['Time-HH'][index] += 12
dotlowell = dotlowell.drop(columns = ['AM-PM'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
import math
for index in range(len(location)):
  if  math.isnan(location['Latitude'][index]):
    if(location['Citation No'][index]==location['Citation No'][index-1]):
      location['Location'][index] = location['Location'][index-1]
      location['Latitude'][index] = location['Latitude'][index-1]
      location['Longitude'][index] = location['Longitude'][index-1]
      location['Zipcode'][index] = location['Zipcode'][index-1]
#fill the Na with previous location if they are teh same person
location.to_excel("/content/drive/MyDrive/CFJJ/Lowell Data Files/Edited | Location Analysis Report - April 2021.xlsx",index = False)

In [ ]:
location = location.drop(columns = ['Date','Time','Offense'])
merge = pd.merge(dotlowell,location, how = 'outer', on = 'Citation No')
merge.to_excel('/content/drive/My Drive/merge.xlsx', index = False)

In [ ]:
import folium
# draw the plot map based on race option 
def race_plot(race):
    m = folium.Map(location=[42.639249,-71.3223647], zoom_start=13)
    colors = ["#FF3300"]

    for _, line in (merge[["Latitude", "Longitude","Race"]]).iterrows():
        lati = line["Latitude"]
        long = line["Longitude"]
        r = line["Race"]
        if not pd.isnull(lati) and r == race:
            folium.Circle(location=(lati, long),
                          radius=150,
                          color=colors[0],
                          opacity=0.0,
                          fill=True,
                          fill_color=colors[0],
                          fill_opacity=0.15).add_to(m)
    return m

In [ ]:
merge = pd.read_excel('/content/drive/MyDrive/CFJJ/Lowell Data Files/merge.xlsx')
merge['Race'].fillna("UNK",inplace = True)
merge.to_excel('/content/drive/MyDrive/CFJJ/Lowell Data Files/merge.xlsx', index = False)

In [ ]:
no_dup = merge.drop_duplicates(subset=['Citation No'])

In [ ]:
race  = pd.DataFrame(no_dup.groupby('Race')['Offense'].count()).rename(columns = {'Offense':'PPL'})
#1219 distinct people
raceppl = race.sort_values(by=['PPL'],ascending=False)
raceppl['percentage/%']= (race['PPL']/1219)*100
raceppl

,PPL,percentage/%
Race,,
WHITE,601,49.302707
HISP,299,24.528302
BLACK,136,11.156686
ASIAN,91,7.465135
UNK,37,3.035275
MIDEST,28,2.296965
AFRICA,10,0.820345
UNKNWN,7,0.574241
INDIAN,6,0.492207


In [ ]:
commonoffense = merge['Detailed Reason'].value_counts()[:6].index.tolist()
commonoffense

['ELECTRONIC DEVICE WHILE OP',
 'SPEEDING',
 'FAIL STOP / STOP SIGN',
 'FAIL STOP / RED LIGHT',
 'SEATBELT VIOLATION',
 'EXCESSIVE WINDOWS TINT']

In [ ]:
merge = pd.read_excel('/content/drive/MyDrive/CFJJ/Team 2/Lowell Data Files/part 1/merge.xlsx')

In [ ]:
raceandcommonoffense = merge[['Race','Detailed Reason']]
#raceandcommonoffense = raceandcommonoffense[raceandcommonoffense['Detailed Reason'].isin(commonoffense)]
raceandcommonoffense = pd.DataFrame(raceandcommonoffense.groupby(['Detailed Reason','Race'])['Race'].count()).rename(columns = {'Race':'PPL'})
raceandcommonoffense = raceandcommonoffense.reset_index()
#overalloffencenum contain total num of ppl for each offense
overalloffencenum = pd.DataFrame(merge[['Detailed Reason','Race']].groupby(['Detailed Reason'])['Race'].count()).rename(columns = {'race':'allPPL'})
raceandcommonoffense = raceandcommonoffense.merge(overalloffencenum, how = 'right',on='Detailed Reason')

In [ ]:
raceandcommonoffense = raceandcommonoffense.rename(columns={'Race_x':'Race', 'Race_y': 'allppl'})
raceandcommonoffense = raceandcommonoffense.drop_duplicates()
raceandcommonoffense['percentage/%'] = raceandcommonoffense['PPL']/raceandcommonoffense['allppl']
raceandcommonoffense.to_excel('/content/drive/MyDrive/CFJJ/Team 2/Lowell Data Files/part 1/offence-race_makeup.xlsx')


In [ ]:
#raceandcommonoffense = raceandcommonoffense.merge(overalloffencenum, how = 'left',on='Detailed Reason')
raceandcommonoffense['allppl'] = raceandcommonoffense.groupby('Detailed Reason')['PPL'].sum()
raceandcommonoffense.columns


Index(['PPL', 'allppl'], dtype='object')

In [ ]:
allppl = raceandcommonoffense['PPL_y']
allppl = allppl.tolist()
allppl

KeyError: ignored

In [ ]:
raceandcommonoffense['allppl/offence'] = allppl
raceandcommonoffense

NameError: ignored

In [ ]:
raceandcommonoffense['allppl/offence'] = allppl
raceandcommonoffense = raceandcommonoffense.drop(columns = ['PPL_y']).rename(columns = {'PPL_x':'PPL'})
raceandcommonoffense['percentage out of race ratio(%)'] = raceandcommonoffense['PPL']*100/raceandcommonoffense['allppl/offence']
raceandcommonoffense = raceandcommonoffense.drop(columns = ['PPL','allppl/offence'])
raceandcommonoffense

NameError: ignored

In [ ]:
raceppl = race.sort_values(by=['PPL'],ascending=False)
raceppl['percentage/%']= (race['PPL']/1219)*100
raceppl

,PPL,percentage/%
Race,,
WHITE,601,49.302707
HISP,299,24.528302
BLACK,136,11.156686
ASIAN,91,7.465135
UNK,46,3.773585
MIDEST,28,2.296965
AFRICA,10,0.820345
INDIAN,6,0.492207
ASPAC,1,0.082034


In [ ]:
severity = pd.DataFrame(merge.groupby('Severity')['Offense'].count()).rename(columns = {'Offense':'PPL'})
severity = serverity.sort_values(by = ['PPL'],ascending = False)
severity['percentage/%']= (severity['PPL']/1972)*100
severity

NameError: ignored

In [8]:
import folium

merge = pd.read_excel('/content/drive/MyDrive/CFJJ/Team 2/Lowell Data Files/part 1/merge.xlsx')

def plot_by_zipcode(df):
    m = folium.Map(location=[42.639249,-71.3223647], zoom_start=13)

    grouped = df[["Latitude", "Longitude", "Severity"]].groupby("Severity")
    

    zipcode_count = len(list(grouped))

    colors = ['#ff9999','#ff0000','#800000']

    zip_index = 0
    for zip_name, group_data in grouped:
        for i, loc in group_data.iterrows():
            lati = loc["Latitude"]
            long = loc["Longitude"]
            if not pd.isnull(lati):
                folium.CircleMarker(location=(lati, long),
                              radius=5,
                              color=colors[zip_index],
                              fill=True,
                              opacity=0.0,
                              fill_color=colors[zip_index],
                              fill_opacity=1,
                              tooltip=zip_name).add_to(m)
        # break
        zip_index += 1
    folium.PolyLine([42.639249,-71.3223647], weight=3, color='blue',opacity=0.8).add_to(m)
    return m

plot_by_zipcode(merge)

Output hidden; open in https://colab.research.google.com to view.